In [23]:
import pandas as pd
import numpy as np
import pytypo
import re
import tensorflow as tf
import tensorflow_addons as tfa
import emoji
from nltk.corpus import stopwords

AUTO = tf.data.experimental.AUTOTUNE

c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Muhammad Ade Aulia\miniconda3\envs\gpu_load\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter 

In [24]:
df_train = pd.read_csv("../data/hapusData/train.csv")
# df_train['Sentimen'] = df_train['Sentimen'].map({'Positif': 1, 'Negatif': 0})
df_train = df_train[['judul', 'narasi', 'label']]
df_train

,judul,narasi,label
0,Puan Maharani Menambahkan Kata Pro Rakyat Dala...,Puan Maharani Tambahkan “Pro Rakyat” di Akhir ...,1
1,Bawa Langsung Lansia ke Rumah Lansia Atmabrata,"Teman2 terkasih, kalau kenal/tahu lansia yg ti...",0
2,Pesan Singkat Perihal Biaya Asuransi Kirim Bar...,"Barang yang dikirim melalui jasa Jalur (JNE), ...",1
3,Ribuan Warga Tasik jalan kaki menuju Jakarta u...,Jalan kaki menuju Jakarta untuk Aksi Kedaulata...,1
4,Breaking News: Megawati Mengundurkan Diri Dari...,Disaat_umat_ISLAM_lagi_gencar_menyerang_Moncon...,1
...,...,...,...
3802,Kanada Sudah Terbebas dari Covid-19,CANADA is now Covid free! i hope philippines n...,1
3803,"Gedung Putih Dibanjiri Massa Pro Ahok, dan men...",BERITA GEMPAR HARI INI!!! PENOLAKAN FPI SAMPAI...,1
3804,Anies Pilih Pembinaan Daripada Cabut KJP Untuk...,Saya tak pernah menggariskan pencabutan (hak) ...,1
3805,Metallica Indonesia Raya,METALLICA… Indonesia Raya….🇮🇩🇮🇩🇮🇩🇮🇩,1


In [25]:
print(df_train.isna().sum())
df_train.dropna(inplace = True)
df_train.isna().sum() , df_train.shape

judul     0
narasi    0
label     0
dtype: int64


(judul     0
 narasi    0
 label     0
 dtype: int64,
 (3807, 3))

In [27]:
df_train = df_train.sample(frac=1,random_state = 0)
df_train.head()

,judul,narasi,label
1410,: PSSI Pecat Ratu Tisha Sebagai Sekjen,"Enggak usah dijelaskan, itu tidak benar",1
2378,Polisi Tangkap Perekayasa Chat Hoax Kapolri da...,Polisi Tangkap Perekayasa Chat Hoax Kapolri da...,0
796,AM Hendropriyono Dirawat di Singapur,MUBAHALAH HABIB RS MANJUR !!\nAM HENDROPRIYONO...,1
2845,Siswa kejang-kejang saat bermain game online d...,Sebuah video memperlihatkan seorang siswa terl...,1
1651,surat kabar luar negeri memberitakan jokowi tu...,Horeeee akhirnya….surat kabar luar negeri memb...,1


In [28]:
def clean(text) : 

  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
  text_cleaning_hash = "#[A-Za-z0-9]+" 
  text_cleaning_num = "(^|\W)\d+"

  text = re.sub(text_cleaning_hash, " ", text).strip()
  text = re.sub(text_cleaning_num, " ", text).strip()
  text = re.sub(text_cleaning_re, " ", text).strip()
  
  text = text.strip()

  out = []
  for word in text.split() :
    # try : 
    #   out.append(word.replace(word, slang[word]))
    # except Exception as e : 
    out.append(word)
      
  return pytypo.correct_sentence(" ".join(out).strip())

In [29]:
def freeEmoji(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [31]:
norm= {" dgn " : " dengan ", ' seller ': ' penjual ',' service ':' pelayanan ', ' tp ':' tapi ', ' recommended ':' rekomendasi ', ' kren ':' keren ', ' kereen ':' keren ', ' mantab ': ' keren ',' matching ':' sesuai ','happy':' senang ','original': 'asli ','ori':'asli ', "trusted" : "terpercaya", "angjaaaassss":"keren", " gue ": " saya ", "bgmn ":" bagaimana ", ' tdk':' tidak ', ' blum ':' belum ', 'mantaaaaaaaappp':' bagus ', 'mantaaap':'bagus ', ' josss ':' bagus ', ' thanks ': ' terima kasih ', 'fast':' cepat ', ' dg ':' dengan ', 'trims':' terima kasih ', 'brg':' barang ', 'gx':' tidak ', ' dgn ':' dengan ', ' recommended':' rekomen ', 'recomend':' rekomen ', 'good':' bagus ', " dgn " : " dengan ", " gue ": " saya ", " dgn ":" dengan ", "bgmn ":" bagaimana ", ' tdk':' tidak ', 
' blum ':' belum ', "quality":"kualitas", 'baguss':'bagus', 'overall' : 'akhirnya', 'mantaaaaaaaappp':' bagus ', ' josss ':' bagus ', ' thanks ': ' terima kasih ', 'fast':' cepat ', 
 'trims':' terima kasih ', 'brg':' barang ', 'gx':' tidak ', ' dgn ':' dengan ', ' real ': ' asli ', ' bnb ': ' baru ' ,
' recommended':' rekomen ', 'recomend':' rekomen ', 'good':'bagus',
'eksis ':'ada ', 'beenilai ':'bernilai ', ' dg ':' dengan ', ' ori ':' asli ', ' setting ':' atur ', " free ":" gratis ",
' yg ':' yang ', 't4 ':'tempat', ' awat ':' awet', ' mantep ':' bagus ', 'mantapp':'bagus', 
'kl ':'kalo', ' k ':' ke ', 'plg ':'pulang ', 'ajah ':'aja ', 'bgt':'banget', 'lbh ':'lebih', 'ayem':'tenang','dsana ':'disana ', 'lg':' lagi',
'pas ':'saat ', ' bnib ': ' baru ', 
' nggak ':' tidak ', 'karna ':'karena ', 'utk ':'untuk ',
' dn ':' dan ', ' mlht ':' melihat ', ' pd ':' pada ', 'mndngr ':'mendengar ', 'crita':'cerita', ' dpt ':' dapat ', ' mksh ':' terima kasih ', ' sellerrrr':' penjual', 'ori ':'asli ', ' new ':' baru ',
'sejrh':'sejarah', 'mnmbh ':'menambah ', 'sayapun':'saya', 'thn ':'tahun ', 'good':'bagus', ' awettt':' awet',
'halu ':'halusinasi ', ' nyantai ':' santai ', 'plus ':'dan ',
' ayang ':' sayang ', ' Rekomendded ':' direkomendasikan ', ' now ': ' sekarang ', 'slalu ':'selalu ', 'photo ': 'foto ', 'slah ':'salah ', 'krn':'karena', ' ga ':' tidak ', 'ok ':'oke ', ' meski':' mesti', ' para ':'parah', ' nawarin':' menawari', 'socmed':'sosial media',
' sya ':' saya ', 'siip':'bagus', ' bny ':' banyak ', ' tdk ':' tidak ', ' byk ':' banyak ', 
' pool ':' sekali ', " pgn ":" ingin ", " gue ":" saya ", " bgmn ":" bagaimana ", " ga ":" tidak ", 
" gak ":" tidak ", " dr ":" dari ", " yg ":" yang ", " lu ":" kamu ", " sya ":" saya ", 
" lancarrr ":" lancar ", " kayak ":" seperti ", " ngawur ":" sembarangan ", " k ":" ke ", 
" luasss ":" luas ", " sy ":" saya ", " thn ":" tahun ", " males ":" malas ",
" tgl ":" tanggal ", " lg ":" lagi ", " bgt ":" banget ",' gua ':' saya ', '\n':' ', ' tpi ':' tapi ', ' standar ':' biasa ', ' standart ': ' biasa ', ' sdh ':' sudah ', ' n ':' dan ', ' gk ': ' tidak ', ' mengecwakan ':' mengecewakan ', ' d ':' di ', ' approved':' setuju', 'ademmmm ':'adem ', ' g ':' tidak ', ' gak ':' tidak ', ' cpt ':' cepat '}

def normalisasi(text):
  for i in norm:
    text = text.replace(i, norm[i])
  return text

def clean(text):
  text = text.strip()
  text = text.lower()
  text = re.sub(r'[^a-zA-Z]+', ' ', text)
  return text

def tokenisasi(text):
    return text.split() 
    
def stopword(text):
    stop_words = set(stopwords.words('indonesian'))
    words = text.split()
    filtered_words = [word for word in words if word.casefold() not in stop_words]
    cleaned_text = ' '.join(filtered_words)
    return cleaned_text

,judul,narasi,label
1410,PSSI Pecat Ratu Tisha Sebagai Sekjen,Enggak usah dijelaskan itu tidak benar,1
2378,Polisi Tangkap Perekayasa Chat Hoax Kapolri da...,Polisi Tangkap Perekayasa Chat Hoax Kapolri da...,0
796,AM Hendropriyono Dirawat di Singapur,MUBAHALAH HABIB RS MANJUR AM HENDROPRIYONO ATI...,1
2845,Siswa kejang kejang saat bermain game online d...,Sebuah video memperlihatkan seorang siswa terl...,1
1651,surat kabar luar negeri memberitakan jokowi tu...,Horee akhirnya surat kabar luar negeri memberi...,1
...,...,...,...
835,Terakhir yang diketahui adalah terhadap Sam Lo...,Ternyata penggunaan ular untuk interogasi oran...,1
3264,Penemuan Sabu di Dalam Nanas Terjadi di Indonesia,Nanas illegal isinya sabu2 yg dikemas Nanas bu...,1
1653,Foto Hati hati kalo kita beli sepatu Logo palu...,Hati hati kalo kita beli sepatu Logo palu arit...,1
2607,Ang Tjoen Min Berada Di Balik Organisasi Teror...,DENSUS NEO CAKRA BIRAWA Ang Tjoeng Ming ipar J...,1
